In [28]:
import pandas as pd
from pivottablejs import pivot_ui
import seaborn as sb
import matplotlib.pyplot as plt  
from ipywidgets import widgets
import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)

%matplotlib inline

sb.set(style="ticks", color_codes=True)
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')
#g = sb.pairplot(df)

In [2]:
df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
#len(df['RESOURCE'].value_counts())
#df['RESOURCE'].plot.hist(bins=7518)

#dfc = df.iloc[:,1:10] 
dfc = df.iloc[:,:]
for col in df.columns:
  dfc[col] = df[col].astype('category')

dfc.describe()


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,2,7518,4243,128,177,449,343,2358,67,343
top,1,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,30872,839,152,21407,4424,1135,4649,6896,10980,4649


In [21]:
#test = pd.pivot_table(dfc,index=['ROLE_TITLE','ROLE_CODE'],aggfunc='count',dropna=True)
#dfc.pivot_table(index='ROLE_TITLE',aggfunc=lambda x:len(x.unique()))
mult = {}
for c in df.columns:
    temp = df.pivot_table(index=c,aggfunc=lambda x:len(x.unique()),fill_value=0).apply(np.max)
    mult[c] = temp

multpd = pd.DataFrame(mult)


In [50]:
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


In [40]:
model = linear_model.LogisticRegression(C=3)
y = df.iloc[:,:1]
X = df.iloc[:,1:9]

y_test = dft.iloc[:,:1]
X_test = dft.iloc[:,1:9]


In [45]:
encoder = preprocessing.OneHotEncoder()
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encoder.fit(np.vstack((X,X_test)))
X = encoder.transform(X)
X_test = encoder.transform(X_test)

In [48]:
print(encoder.n_values_)
print(encoder.feature_indices_)

[312154 311780 311179 286792 286793 311868 311868 308575]
[      0  312154  623934  935113 1221905 1508698 1820566 2132434 2441009]


In [ ]:
mean_auc = 0.0
n = 10  # repeat the CV procedure 10 times to get more precise results
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
        X, y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]

    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print "AUC (fold %d/%d): %f" % (i + 1, n, roc_auc)
    mean_auc += roc_auc

    print "Mean AUC: %f" % (mean_auc/n)

# === Predictions === #
# When making predictions, retrain the model on the whole training set
model.fit(X, y)
preds = model.predict_proba(X_test)[:, 1]
filename = raw_input("Enter name for submission file: ")
save_results(preds, filename + ".csv")

MGR_ID  RESOURCE  ROLE_DEPTNAME  ROLE_FAMILY  \
ROLE_TITLE ROLE_CODE                                                 
117879     117880       1256      1256           1256         1256   
           117888        NaN       NaN            NaN          NaN   
           117898        NaN       NaN            NaN          NaN   
           117900        NaN       NaN            NaN          NaN   
           117908        NaN       NaN            NaN          NaN   
           117948        NaN       NaN            NaN          NaN   
           117973        NaN       NaN            NaN          NaN   
           117987        NaN       NaN            NaN          NaN   
           118030        NaN       NaN            NaN          NaN   
           118046        NaN       NaN            NaN          NaN   
           118049        NaN       NaN            NaN          NaN   
           118055        NaN       NaN            NaN          NaN   
           118132        NaN       NaN            NaN          NaN   
           118175        NaN       NaN            NaN          NaN   
           118196        NaN       NaN            NaN          NaN   
           118206        NaN       NaN            NaN          NaN   
           118209        NaN       NaN            NaN          NaN   
           118232        NaN       NaN            NaN          NaN   
           118261        NaN       NaN            NaN          NaN   
           118276        NaN       NaN            NaN          NaN   
           118279        NaN       NaN            NaN          NaN   
           118296        NaN       NaN            NaN          NaN   
           118319        NaN       NaN            NaN          NaN   
           118322        NaN       NaN            NaN          NaN   
           118332        NaN       NaN            NaN          NaN   
           118364        NaN       NaN            NaN          NaN   
           118373        NaN       NaN            NaN          NaN   
           118399        NaN       NaN            NaN          NaN   
           118425        NaN       NaN            NaN          NaN   
           118454        NaN       NaN            NaN          NaN   
...                      ...       ...            ...          ...   
311867     158291        NaN       NaN            NaN          NaN   
           159118        NaN       NaN            NaN          NaN   
           159680        NaN       NaN            NaN          NaN   
           159789        NaN       NaN            NaN          NaN   
           161101        NaN       NaN            NaN          NaN   
           162862        NaN       NaN            NaN          NaN   
           163313        NaN       NaN            NaN          NaN   
           163732        NaN       NaN            NaN          NaN   
           166594        NaN       NaN            NaN          NaN   
           166801        NaN       NaN            NaN          NaN   
           169635        NaN       NaN            NaN          NaN   
           174393        NaN       NaN            NaN          NaN   
           180928        NaN       NaN            NaN          NaN   
           184276        NaN       NaN            NaN          NaN   
           187169        NaN       NaN            NaN          NaN   
           188048        NaN       NaN            NaN          NaN   
           192869        NaN       NaN            NaN          NaN   
           208127        NaN       NaN            NaN          NaN   
           208567        NaN       NaN            NaN          NaN   
           209875        NaN       NaN            NaN          NaN   
           212194        NaN       NaN            NaN          NaN   
           216827        NaN       NaN            NaN          NaN   
           239004        NaN       NaN            NaN          NaN   
           240105        NaN       NaN            NaN          NaN   
           247660        NaN    

In [83]:
role_title_table.dtypes

col_0
count    int64
dtype: object

In [86]:
row_title_table.*

SyntaxError: invalid syntax (<ipython-input-86-1ac8b4367f55>, line 1)

Looks like Role Title and Role Code are similar 
Use PCA ? from sklearn.decompisition import PCA


AUC = Logistic Regression returns the probability that a certain input features would be in a certain class
ROC curve visualizaes all possible thresholds
Misclassification rate is error rate for a single threshold